# BEE 4750 Homework 2: Dissolved Oxygen

**Name**: Sinahy Fragoso-De la Paz

**ID**: sf435

> **Due Date**
>
> Friday, 09/22/23, 9:00pm

## Overview

### Instructions

This assignment asks you to use a simulation model for dissolved oxygen
to assess the impacts of two wastewater streams, including minimum
treatment levels and the impact of uncertain environmental conditions.
You will also be asked to identify a minimum distance for the addition
of a third discharge stream.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [127]:
import Pkg
Pkg.add("Plots")
Pkg.add("LaTeXStrings")
Pkg.add("Measures")
Pkg.add("LinearAlgebra")
Pkg.activate(@__DIR__)
Pkg.instantiate()

   Resolving package versions...


  No Changes to `~/Downloads/hw02-sinahy-frag-1/Project.toml`
  No Changes to `~/Downloads/hw02-sinahy-frag-1/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Downloads/hw02-sinahy-frag-1/Project.toml`
  No Changes to `~/Downloads/hw02-sinahy-frag-1/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Downloads/hw02-sinahy-frag-1/Project.toml`
  No Changes to `~/Downloads/hw02-sinahy-frag-1/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Downloads/hw02-sinahy-frag-1/Project.toml`
  No Changes to `~/Downloads/hw02-sinahy-frag-1/Manifest.toml`


  Activating project at `~/Downloads/hw02-sinahy-frag-1`


In [128]:
using Plots
using LaTeXStrings
using Measures
using Random
Random.seed!(1)
using LinearAlgebra
using Distributions

## Problems (Total: 40 Points)

A river which flows at 6 km/d is receiving waste discharges from two
sources which are 15 km apart, as shown in
<a href="#fig-river" class="quarto-xref">Figure 1</a>. The oxygen
reaeration rate is 0.55 day<sup>-1</sup>, and the decay rates of CBOD
and NBOD are are 0.55 and 0.25 day<sup>-1</sup>, respectively. The
river’s saturated dissolved oxygen concentration is 10m g/L.

![Figure 1: Schematic of the
system](attachment:figures/river_diagram.png)

### Problem 1 (8 points)

If the characteristics of the river inflow and waste discharges are
given in <a href="#tbl-river" class="quarto-xref">Table 1</a>, write a
Julia model to compute the dissolved oxygen concentration from the first
wastewater discharge to an arbitrary distance `d` km downstream. Use
your model to compute the maximum dissolved oxygen concentration up to
50km downstream and how far downriver this maximum occurs.

|    Parameter     |            River Inflow |         Waste Stream 1 |         Waste Stream 2 |
|:----------------:|------------------------:|-----------------------:|-----------------------:|
|      Inflow      | 100,000 m<sup>3</sup>/d | 10,000 m<sup>3</sup>/d | 15,000 m<sup>3</sup>/d |
| DO Concentration |                7.5 mg/L |                 5 mg/L |                 5 mg/L |
|       CBOD       |                  5 mg/L |                50 mg/L |                45 mg/L |
|       NBOD       |                  5 mg/L |                35 mg/L |                35 mg/L |

Table 1: River inflow and waste stream characteristics for Problem 1.

In [129]:
#calculate inital concentration values
Q_inflow = 100000.0  # m³/d
Q_waste1 = 10000.0   # m³/d
Q_waste2 = 15000.0   # m³/d
DO_inflow = 7.5      # mg/L
DO_waste1 = 5.0      # mg/L
DO_waste2 = 5.0      # mg/L
CBOD_inflow = 5.0    # mg/L
CBOD_waste1 = 50.0   # mg/L
CBOD_waste2 = 45.0   # mg/L
NBOD_inflow = 5.0    # mg/L
NBOD_waste1 = 35.0   # mg/L
NBOD_waste2 = 35.0   # mg/L

C0_waste1 = ((DO_inflow * Q_inflow) + (DO_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
B0_waste1 = ((CBOD_inflow * Q_inflow) + (CBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1) 
N0_waste1 = ((NBOD_inflow * Q_inflow) + (NBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
C0_waste2 = ((DO_inflow * Q_inflow) + (DO_waste2 * Q_waste2))/(Q_inflow + Q_waste2)
B0_waste2 = ((CBOD_inflow * Q_inflow) + (CBOD_waste2 * Q_waste2))/(Q_inflow + Q_waste2) 
N0_waste2 = ((NBOD_inflow * Q_inflow) + (NBOD_waste2 * Q_waste2))/(Q_inflow + Q_waste2)

println("$C0_waste1")
println("$B0_waste1")
println("$N0_waste1")
println("$C0_waste2")
println("$B0_waste2")
println("$N0_waste2")

7.2727272727272725
9.090909090909092
7.7272727272727275
7.173913043478261
10.217391304347826
8.91304347826087


In [130]:
#USES STREAM 1 with chat corrections

Q_inflow = 100000.0  # m³/d
Q_waste1 = 10000.0   # m³/d
DO_inflow = 7.5      # mg/L
DO_waste1 = 5.0      # mg/L
CBOD_inflow = 5.0    # mg/L
CBOD_waste1 = 50.0   # mg/L
NBOD_inflow = 5.0    # mg/L
NBOD_waste1 = 35.0   # mg/L

#calculate inital concentration values
C0_waste1 = ((DO_inflow * Q_inflow) + (DO_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
B0_waste1 = ((CBOD_inflow * Q_inflow) + (CBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1) 
N0_waste1 = ((NBOD_inflow * Q_inflow) + (NBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1)

println("$C0_waste1")
println("$B0_waste1")
println("$N0_waste1")

Cs = 10              # mg/L
ka = .55             # 1/day
kc = .55             # 1/day
kn = .25             # 1/day
U = 10               # mg/L

function calculate_do_concentration(d, Cs, C0_waste1, B0_waste1, N0_waste1, ka, kc, kn, U)
    a1 = exp(-ka * d / U)
    a2 = (kc / (ka-kc)) * (exp(-kc * d / U) - a1)
    a3 = (kn / (ka-kn)) * (exp(-kn * d / U) - a1)
    C1 = (Cs * (1 - a1)) + (C0_waste1 - a1) - (B0_waste1 - a2) - (N0_waste1 - a3) 
    return C1
end

d = 0:0.1:50
Cx1 = (y1 -> calculate_do_concentration(y1, Cs, C0_waste1, B0_waste1, N0_waste1, ka, kc, kn, U)).(d)

#plot Cx1
plot(d, C; linewidth=3, label="Dissolved Oxygen", tickfontsize=16, guidefontsize= 16, legendfontsize=16)
xlabel!(L"$x$ (km)")
ylabel!("DO concentration (mg/L)")
hline!([3], color=:red,
    linestyle=:dash,
    label="Regulatory Standard")
plot!(size=(500, 500))

d_current = 0.0  #current distance downstream
DO_concentration = DO_inflow  #initial DO concentration
    
    while d_current < d
        #rearation
        r = ka(Cs-C1)
        #calculate the change in DO concentration due to CBOD and NBOD
        CBOD = kc * B0_waste1 * exp(-kc*d/U)
        NBOD = kn * N0_waste1 * exp(-kn*d/U)
        
        #calculate the net change in DO concentration
        DO_change = r - CBOD - NBOD

        C1 = (Cs * (1 - a1)) + (C0_waste1 - a1) - (B0_waste1 - a2) - (N0_waste1 - a3) 

        
        #update the DO concentration and distance downstream
        DO_concentration += DO_change
        d_current += 1.0  # Move downstream by 1 km
        
        #ensure DO concentration does not go below 0 mg/L
        #DO_concentration = max(0.0, DO_concentration)
    end
    
    return DO_concentration
end

#calculate maximum DO concentration and distance downstream
max_DO = maximum(Cx1)
max_d = d[argmax(Cx1)]

#for d in 1:50  #check up to 50 km downstream
    #current_DO = calculate_do_concentration(d_current)
    #if current_DO > max_DO
        #max_DO = current_DO
        #max_d = d
   #end
#end

println("Maximum dissolved oxygen concentration: ", max_DO, " mg/L")
println("Distance downstream where maximum occurs: ", max_d, " km")


7.2727272727272725
9.090909090909092
7.7272727272727275


MethodError: MethodError: no method matching isless(::Float64, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64})

Closest candidates are:
  isless(::T, !Matched::T) where T<:Union{Float16, Float32, Float64}
   @ Base float.jl:547
  isless(!Matched::AbstractVector, ::AbstractVector)
   @ Base abstractarray.jl:2913
  isless(::Real, !Matched::AbstractGray)
   @ ColorTypes ~/.julia/packages/ColorTypes/1dGw6/src/operations.jl:39
  ...


In [131]:
#USES STREAM 1

Q_inflow = 100000.0  # m³/d
Q_waste1 = 10000.0   # m³/d
DO_inflow = 7.5      # mg/L
DO_waste1 = 5.0      # mg/L
CBOD_inflow = 5.0    # mg/L
CBOD_waste1 = 50.0   # mg/L
NBOD_inflow = 5.0    # mg/L
NBOD_waste1 = 35.0   # mg/L

#calculate inital concentration values
C0_waste1 = ((DO_inflow * Q_inflow) + (DO_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
B0_waste1 = ((CBOD_inflow * Q_inflow) + (CBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1) 
N0_waste1 = ((NBOD_inflow * Q_inflow) + (NBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1)

println("$C0_waste1")
println("$B0_waste1")
println("$N0_waste1")

Cs = 10              # mg/L
ka = .55             # 1/day
kc = .55             # 1/day
kn = .25             # 1/day
U = 10               # mg/L

function calculate_do_concentration(d, Cs, C0_waste1, B0_waste1, 
N0_waste1, ka, kc, kn, U)
    a1 = exp(-ka * d / U)
    a2 = (kc / (ka-kc)) * (exp(-kc * d / U) - a1)
    a3 = (kn / (ka-kn)) * (exp(-kn * d / U) - a1)
    C1 = (Cs * (1 - a1)) + (C0_waste1 - a1) - (B0_waste1 - a2) - (N0_waste1 - a3) 
    return C1
end

d = 0:0.1:50
Cx1 = (y1 -> calculate_do_concentration(y1, Cs, C0_waste1, B0_waste1, N0_waste1, ka, kc, kn, U)).(d)

#plot Cx1
plot(d, C; linewidth=3, label="Dissolved Oxygen", tickfontsize=16, guidefontsize= 16, legendfontsize=16)
xlabel!(L"$x$ (km)")
ylabel!("DO concentration (mg/L)")
hline!([3], color=:red,
    linestyle=:dash,
    label="Regulatory Standard")
plot!(size=(500, 500))

d_current = 0.0  #current distance downstream
DO_concentration = DO_inflow  #initial DO concentration
    
    while d_current < d
        #rearation
        r = ka(Cs-C1)
        #calculate the change in DO concentration due to CBOD and NBOD
        CBOD = kc * B0_waste1 * exp(-kc*d/U)
        NBOD = kn * N0_waste1 * exp(-kn*d/U)
        
        #calculate the net change in DO concentration
        DO_change = r - CBOD - NBOD

        C1 = (Cs * (1 - a1)) + (C0_waste1 - a1) - (B0_waste1 - a2) - (N0_waste1 - a3) 

        
        #update the DO concentration and distance downstream
        DO_concentration += DO_change
        d_current += 1.0  # Move downstream by 1 km
        
        #ensure DO concentration does not go below 0 mg/L
        DO_concentration = max(0.0, DO_concentration)
    end
    
    return DO_concentration
end

#calculate maximum DO concentration and distance downstream
max_DO = 0.0
max_d = 0.0

for d in 1:50  #check up to 50 km downstream
    current_DO = calculate_do_concentration(d_current)
    if current_DO > max_DO
        max_DO = current_DO
        max_d = d
    end
end

println("Maximum dissolved oxygen concentration: ", max_DO, " mg/L")
println("Distance downstream where maximum occurs: ", max_d, " km")


7.2727272727272725
9.090909090909092
7.7272727272727275


MethodError: MethodError: no method matching isless(::Float64, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64})

Closest candidates are:
  isless(::T, !Matched::T) where T<:Union{Float16, Float32, Float64}
   @ Base float.jl:547
  isless(!Matched::AbstractVector, ::AbstractVector)
   @ Base abstractarray.jl:2913
  isless(::Real, !Matched::AbstractGray)
   @ ColorTypes ~/.julia/packages/ColorTypes/1dGw6/src/operations.jl:39
  ...


In [132]:
#USES BOTH STREAMS


#calculate inital concentration values
Q_inflow = 100000.0  # m³/d
Q_waste1 = 10000.0   # m³/d
Q_waste2 = 15000.0   # m³/d
DO_inflow = 7.5      # mg/L
DO_waste1 = 5.0      # mg/L
DO_waste2 = 5.0      # mg/L
CBOD_inflow = 5.0    # mg/L
CBOD_waste1 = 50.0   # mg/L
CBOD_waste2 = 45.0   # mg/L
NBOD_inflow = 5.0    # mg/L
NBOD_waste1 = 35.0   # mg/L
NBOD_waste2 = 35.0   # mg/L

C0_waste1 = ((DO_inflow * Q_inflow) + (DO_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
B0_waste1 = ((CBOD_inflow * Q_inflow) + (CBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1) 
N0_waste1 = ((NBOD_inflow * Q_inflow) + (NBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
C0_waste2 = ((DO_inflow * Q_inflow) + (DO_waste2 * Q_waste2))/(Q_inflow + Q_waste2)
B0_waste2 = ((CBOD_inflow * Q_inflow) + (CBOD_waste2 * Q_waste2))/(Q_inflow + Q_waste2) 
N0_waste2 = ((NBOD_inflow * Q_inflow) + (NBOD_waste2 * Q_waste2))/(Q_inflow + Q_waste2)

println("$C0_waste1")
println("$B0_waste1")
println("$N0_waste1")
println("$C0_waste2")
println("$B0_waste2")
println("$N0_waste2")

Cs = 10              # mg/L
ka = .55             # 1/day
kc = .55             # 1/day
kn = .25             # 1/day
U = 10               # mg/L

function calculate_do_concentration(d, Cs, C0_waste1, B0_waste1, 
N0_waste1, C0_waste2, B0_waste2, N0_waste2, ka, kc, kn, U)
    a1 = exp(-ka * d / U)
    a2 = (kc / (ka-kc)) * (exp(-kc * d / U) - a1)
    a3 = (kn / (ka-kn)) * (exp(-kn * d / U) - a1)
    C1 = (Cs * (1 - a1)) + (C0_waste1 - a1) - (B0_waste1 - a2) - (N0_waste1 - a3) 
    C2 = (Cs * (1 - a1)) + (C0_waste2 - a1) - (B0_waste2 - a2) - (N0_waste2 - a3) 
    return C1
    return C2
end

d = 0:0.1:50
Cx1 = (y1 -> calculate_do_concentration(y1, Cs, C0_waste1, B0_waste1, N0_waste1, ka, kc, kn, U)).(d)
Cx2 = (y2 -> calculate_do_concentration(y2, Cs, C0_waste2, B0_waste2, N0_waste2, ka, kc, kn, U)).(d)

    
plot(d, C; linewidth=3, label="Dissolved Oxygen", tickfontsize=16, guidefontsize= 16, legendfontsize=16)
xlabel!(L"$x$ (km)")
ylabel!("DO concentration (mg/L)")
hline!([3], color=:red,
    linestyle=:dash,
    label="Regulatory Standard")
plot!(size=(500, 500))

    # Initialize variables
d_current = 0.0  # Current distance downstream
DO_concentration = DO_inflow  # Initial DO concentration
    
    while d_current < d
        # Calculate the change in DO concentration due to waste streams
        DO_change_waste1 = (Q_waste1 / Q_inflow) * (DO_inflow - DO_waste1)
        DO_change_waste2 = (Q_waste2 / Q_inflow) * (DO_inflow - DO_waste2)
        
        # Calculate the change in DO concentration due to CBOD and NBOD
        DO_change_CBOD = (Q_inflow / 1000000.0) * (CBOD_inflow - CBOD_waste1 - CBOD_waste2)
        DO_change_NBOD = (Q_inflow / 1000000.0) * (NBOD_inflow - NBOD_waste1 - NBOD_waste2)
        
        # Calculate the net change in DO concentration
        DO_change = DO_change_waste1 + DO_change_waste2 - DO_change_CBOD - DO_change_NBOD
        
        # Update the DO concentration and distance downstream
        DO_concentration += DO_change
        d_current += 1.0  # Move downstream by 1 km
        
        # Ensure DO concentration does not go below 0 mg/L
        DO_concentration = max(0.0, DO_concentration)
    end
    
    return DO_concentration
end

# Calculate maximum DO concentration and distance downstream
max_DO = 0.0
max_d = 0.0

for d in 1:50  # Check up to 50 km downstream
    current_DO = calculate_do_concentration(d)
    if current_DO > max_DO
        max_DO = current_DO
        max_d = d
    end
end

println("Maximum dissolved oxygen concentration: ", max_DO, " mg/L")
println("Distance downstream where maximum occurs: ", max_d, " km")


7.2727272727272725
9.090909090909092
7.7272727272727275
7.173913043478261
10.217391304347826
8.91304347826087


MethodError: MethodError: no method matching isless(::Float64, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64})

Closest candidates are:
  isless(::T, !Matched::T) where T<:Union{Float16, Float32, Float64}
   @ Base float.jl:547
  isless(!Matched::AbstractVector, ::AbstractVector)
   @ Base abstractarray.jl:2913
  isless(::Real, !Matched::AbstractGray)
   @ ColorTypes ~/.julia/packages/ColorTypes/1dGw6/src/operations.jl:39
  ...


In [133]:


function calculate_dissolved_oxygen_concentration(d::Float64)
    # Constants
    Q_inflow = 100000.0  # m³/d
    Q_waste1 = 10000.0   # m³/d
    Q_waste2 = 15000.0   # m³/d
    DO_inflow = 7.5      # mg/L
    DO_waste1 = 5.0      # mg/L
    DO_waste2 = 5.0      # mg/L
    CBOD_inflow = 5.0    # mg/L
    CBOD_waste1 = 50.0   # mg/L
    CBOD_waste2 = 45.0   # mg/L
    NBOD_inflow = 5.0    # mg/L
    NBOD_waste1 = 35.0   # mg/L
    NBOD_waste2 = 35.0   # mg/L
    Cs = 10              # mg/L
    ka = .55             # 1/day
    kc = .55             # 1/day
    kn = .25             # 1/day
    U = 10               # mg/L
    C0_waste1 = ((DO_inflow * Q_inflow) + (DO_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
    B0_waste1 = ((CBOD_inflow * Q_inflow) + (CBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1) 
    N0_waste1 = ((NBOD_inflow * Q_inflow) + (NBOD_waste1 * Q_waste1))/(Q_inflow + Q_waste1)
    C0_waste2 = ((DO_inflow * Q_inflow) + (DO_waste2 * Q_waste2))/(Q_inflow + Q_waste2)
    B0_waste2 = ((CBOD_inflow * Q_inflow) + (CBOD_waste2 * Q_waste1))/(Q_inflow + Q_waste2) 
    N0_waste2 = ((NBOD_inflow * Q_inflow) + (NBOD_waste2 * Q_waste2))/(Q_inflow + Q_waste2)
    
    # Initialize variables
    d_current = 0.0  # Current distance downstream
    DO_concentration = DO_inflow  # Initial DO concentration
    
    while d_current < d
        # Calculate the change in DO concentration due to waste streams
        DO_change_waste1 = (Q_waste1 / Q_inflow) * (DO_inflow - DO_waste1)
        DO_change_waste2 = (Q_waste2 / Q_inflow) * (DO_inflow - DO_waste2)
        
        # Calculate the change in DO concentration due to CBOD and NBOD
        DO_change_CBOD = (Q_inflow / 1000000.0) * (CBOD_inflow - CBOD_waste1 - CBOD_waste2)
        DO_change_NBOD = (Q_inflow / 1000000.0) * (NBOD_inflow - NBOD_waste1 - NBOD_waste2)
        
        # Calculate the net change in DO concentration
        DO_change = DO_change_waste1 + DO_change_waste2 - DO_change_CBOD - DO_change_NBOD
        
        # Update the DO concentration and distance downstream
        DO_concentration += DO_change
        d_current += 1.0  # Move downstream by 1 km
        
        # Ensure DO concentration does not go below 0 mg/L
        DO_concentration = max(0.0, DO_concentration)
    end
    
    return DO_concentration
end

# Calculate maximum DO concentration and distance downstream
max_DO = 0.0
max_d = 0.0

for d in 1:50  # Check up to 50 km downstream
    current_DO = calculate_dissolved_oxygen_concentration(d)
    if current_DO > max_DO
        max_DO = current_DO
        max_d = d
    end
end

println("Maximum dissolved oxygen concentration: ", max_DO, " mg/L")
println("Distance downstream where maximum occurs: ", max_d, " km")


#compute dissolved oxygen concentrations 
#function do_concentration(x, Cs, C0, B0, N0, ka, kc, kn, U)

MethodError: MethodError: no method matching calculate_dissolved_oxygen_concentration(::Int64)

Closest candidates are:
  calculate_dissolved_oxygen_concentration(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any)
   @ Main ~/Downloads/hw02-sinahy-frag-1/hw02.ipynb:8
  calculate_dissolved_oxygen_concentration(!Matched::Float64)
   @ Main ~/Downloads/hw02-sinahy-frag-1/hw02.ipynb:3


### Problem 2 (4 points)

Use your model to plot the dissolved oxygen concentration in the river
from the first waste stream to 50km downstream. What do you notice?

### Problem 3 (3 points)

Under the assumptions of Problem 1, determine the distance from waste
stream 2 it will take for the dissolved oxygen concentration of the
river to recover to 6 mg/L.

### Problem 4 (5 points)

What is the minimum level of treatment (% removal of organic waste) for
waste stream 2 that will ensure that the dissolved oxygen concentration
never drops below 4 mg/L, assuming that waste stream 1 remains
untreated?

### Problem 5 (5 points)

If both waste streams are treated equally, what is the minimum level of
treatment (% removal of organic waste) for the two sources required to
ensure that the dissolved oxygen concentration never drops below 4 mg/L?

### Problem 6 (5 points)

Suppose you are responsible for designing a waste treatment plan for
discharges into the river, with a regulatory mandate to keep the
dissolved oxygen concentration above 4 mg/L. Discuss whether you’d opt
to treat waste stream 2 alone or both waste streams equally. What other
information might you need to make a conclusion, if any?

### Problem 7 (5 points)

Suppose that it is known that the DO concentrations at the river inflow
can vary uniformly between 6 mg/L and 8 mg/L. How often will the
treatment plan identified in Problem 5 (both waste streams treated
equally) fail to comply with the regulatory standard?

### Problem 8 (5 points)

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

Assume that the treatment plan you identified in Problem 5 is still in
place for the existing discharges. If the third discharge will not be
treated, under the original inflow conditions (7.5 mg/L DO), how far
downstream from the second discharge does this third discharge need to
be placed to keep the river concentration from dropping below 4 mg/L?

## References

List any external references consulted, including classmates.

GeeksforGeeks. (2020, March 26). Julia global keyword: Creating a global variable in julia. GeeksforGeeks. https://www.geeksforgeeks.org/julia-global-keyword-creating-a-global-variable-in-julia/# 